This notebook uses the subsetted GHM by the 4 anthromes to calculate the percentage of cell from the master grid that is encroached. 
This is only for the terrestrial environment

In [1]:
import arcpy
from arcpy.sa import *

In [2]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure"
ghm = "gHM_extended_maskGADM36_cea_20191127.tif"
grid = "..\\Human_pressure_outputs.gdb\\biodiversity_facets_dissolved"

In [3]:
ZSaTGrid = "..\\Human_pressure_outputs.gdb\\ZSaTGrid"

Zonal Statistics calculates the `COUNT` by default. 

In [4]:
outZSaT = ZonalStatisticsAsTable(grid,
                                 "CELL_ID", 
                                 ghm,
                                 ZSaTGrid, "DATA", "ALL")

In [10]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\\Human_pressure_outputs.gdb"
ghm_class_rasters = arcpy.ListRasters("gHM*")
for raster in ghm_class_rasters:
    print(raster)

gHM_Urban
gHM_Rainfed
gHM_Irrigated
gHM_Rangeland


In [15]:
for raster in ghm_class_rasters:
    outTableName = "..\\Human_pressure_outputs.gdb\\ZSat_" + raster.split("_")[1]
    outZSat = ZonalStatisticsAsTable(grid,
                                     "CELL_ID", 
                                     raster,
                                     outTableName, "DATA", "ALL")

- Merge the tables 5 tables should do mapping fields, keeping only the appropriate fields and renaming accordingly. 
- Then the `None` values should be replace by `0`.
- Finally calculate the proportions

In [35]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\\Human_pressure_outputs.gdb"
zsa_tables = arcpy.ListTables("ZSat_*")
for tbl in zsa_tables:
    print(tbl)

ZSat_Urban
ZSat_Rainfed
ZSat_Irrigated
ZSat_Rangeland


In [41]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\Human_pressure_outputs.gdb"
zsa_tables = arcpy.ListTables("ZSat_*")
ZSaTGrid = "ZSaTGrid"
for tbl in zsa_tables:
    pos = len(arcpy.ListFields(ZSaTGrid))
    arcpy.management.JoinField(ZSaTGrid, "CELL_ID", tbl, "CELL_ID", "COUNT")
    newField_name = "COUNT_" + tbl.split("_")[1]
    oldField_name = arcpy.ListFields(ZSaTGrid)[pos].name
    print("old: " + oldField_name)
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    print("new: " + newField_name)

old: COUNT_1
new: COUNT_Urban
old: COUNT_1
new: COUNT_Rainfed
old: COUNT_1
new: COUNT_Irrigated
old: COUNT_1
new: COUNT_Rangeland


In [42]:
for field in arcpy.ListFields(ZSaTGrid):
    print(field.name)

OBJECTID
CELL_ID
ZONE_CODE
COUNT
AREA
MIN
MAX
RANGE
MEAN
STD
SUM
COUNT_Urban
COUNT_Rainfed
COUNT_Irrigated
COUNT_Rangeland


We have to replace the `None` with `0`s.

In [43]:
fieldObs = arcpy.ListFields(ZSaTGrid)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [44]:
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU

We have to create new fields (one for each type of encroachment) and calculate the proportion. 

In [45]:
for tbl in zsa_tables:
    prop_name = "prop_" + tbl.split("_")[1]
    COUNT_name = "COUNT_" + tbl.split("_")[1]
    py_formula = "(!" + COUNT_name + "! / !COUNT!) * 100"
    print(prop_name + "\n" + COUNT_name + "\n" + py_formula) 

prop_Urban
COUNT_Urban
(!COUNT_Urban! / !COUNT!) * 100
prop_Rainfed
COUNT_Rainfed
(!COUNT_Rainfed! / !COUNT!) * 100
prop_Irrigated
COUNT_Irrigated
(!COUNT_Irrigated! / !COUNT!) * 100
prop_Rangeland
COUNT_Rangeland
(!COUNT_Rangeland! / !COUNT!) * 100


In [46]:
for tbl in zsa_tables:
    prop_name = "prop_" + tbl.split("_")[1]
    arcpy.management.AddField(ZSaTGrid, 
                          prop_name, 
                          "DOUBLE", 
                          None, None, None, 
                          prop_name, "NULLABLE", "NON_REQUIRED", '')
    COUNT_name = "COUNT_" + tbl.split("_")[1]
    py_formula = "(!" + COUNT_name + "! / !COUNT!) * 100"
    arcpy.management.CalculateField(ZSaTGrid, 
                                prop_name, 
                                py_formula, 
                                "PYTHON3", '')
    print(prop_name + "\n" + COUNT_name + "\n" + py_formula) 

prop_Urban
COUNT_Urban
(!COUNT_Urban! / !COUNT!) * 100
prop_Rainfed
COUNT_Rainfed
(!COUNT_Rainfed! / !COUNT!) * 100
prop_Irrigated
COUNT_Irrigated
(!COUNT_Irrigated! / !COUNT!) * 100
prop_Rangeland
COUNT_Rangeland
(!COUNT_Rangeland! / !COUNT!) * 100


In [47]:
arcpy.TableToTable_conversion(ZSaTGrid, "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure", "human_prop.csv")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\human_prop.csv'>

The table has been uploaded to arcgis online via the browser. 